# Aula 09 - Fluxo Ótico

Exemplo adaptado do link: https://docs.opencv.org/3.4/d7/d8b/tutorial_py_lucas_kanade.html

Referências: 
    1. Szeilisk - Cap 8.4
    1. Artigo Bouguet - Pyramidal Implementation of the Lucas Kanade Feature Tracker


In [16]:
%reset -f
%matplotlib inline

#!pip install opencv-python

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

import time

print(cv.__version__)

3.1.0


___

1. Realizar o download das imagens no Blackboard e verificar se o notebook consegue abri-las:

___

2. Detectar alguns pontos interessantes nela usando o método visto na aula passada.

___

3. Agora vamos realizar o tracking dos pontos usando fluxo ótico. 

  * Ver o artigo do Bouguet no Blackboard para mais detalhes.
  * Fonte das imagens: http://www.cvg.reading.ac.uk/PETS2009/a.html

## Projeto 2-1: Estabilização de imagens

**Motivação**: https://www.youtube.com/watch?v=4vt7bGEen2s

Agora que você consegue capturar o fluxo ótico entre duas imagens consecultivas, vamos utilizá-lo de modo inverso: como seria uma forma de compensar eventuais oscilações na câmera?

Projete um programa que captura as imagens da webcam e realiza a estabilização da imagem. Você notará que se utilizar o programa acima, a estabilização será parcial e falha. Por que?

Você deve construir um Jupyter Notebook que utiliza o Dense Optical Flow para corrigir o problema acima. O notebook deve conter comentários acerca da solução usada.

O programa base para uso da webcam:

```python
captura = cv2.VideoCapture(0)

# Para não deixar encavalar os frames
captura.set(cv2.CAP_PROP_BUFFERSIZE, 1)
 
while(1):
    ret, frame = captura.read()
    cv2.imshow("Video", frame)
   
    # Pressione ESC para sair do loop
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    time.sleep(0.5)
 
captura.release()
cv2.destroyAllWindows()
```

### Entrega: 12/Set 23:59 via GitHub.

### Rubrica do Projeto:

    I. Não entregou ou entregou apenas um rascunho.
    D. O programa faz apenas uma estabilização parcial com os programas da Aula 09.
    C. Utiliza o Dense Optical Flow de uma janela no centro da imagem.
    B. Utiliza meios para compensar as faixas pretas nos cantos da imagem com algum limite.
    A. Consegue realizar a compensação em rotação no eixo de profundidade.
    
    +1/2 Conceito para implementações que comprovadamente melhoram o desempenho da estabilização.
    -1/2 Conceito se o notebook não contiver uma explicação detalhada da solução apresentada.

In [17]:
def cut_window(img, compensation, frame):
    rows,columns =  img.shape
    print(compensation)
    rows_init = int(rows*3 / 8)
    columns_init = int(columns*3 / 8)
    
    rows_end = int(rows * 5/ 8)
    columns_end = int(columns * 5 / 8)

    
    
    rows_init += compensation[1]
    rows_end += compensation[1]
    columns_init += compensation[0]
    columns_end += compensation[0]
    
    #checar pra n dar erro
        
    
    upper_left = (columns_init,rows_init)
    bottom_right = (columns_end, rows_end)
    

    cv2.rectangle(frame, upper_left, bottom_right, (0, 255, 0), 2)
    
    cropped = img[rows_init : rows_end, columns_init : columns_end]


    return cropped

    

In [18]:
'''
 Based on the following tutorial:
   http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_video/py_lucas_kanade/py_lucas_kanade.html
'''

import numpy as np
import cv2

# Start the webcam
cap = cv2.VideoCapture(0)
# cap = cv2.VideoCapture('../data/vtest.avi')

# Take the first frame and convert it to gray
ret, frame = cap.read()
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

acumulator_x = 0
acumulator_y = 0

gray = cut_window(gray,[acumulator_x, acumulator_y], frame)


# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('normal_frame.avi',fourcc, 20.0, (640,480))
out1 = cv2.VideoWriter('cropped_frame.avi',fourcc, 20.0, (640,480))
out2 = cv2.VideoWriter('estabilized_frame.avi',fourcc, 20.0, (640,480))
out3 = cv2.VideoWriter('final_estabilization.avi',fourcc, 20.0, (640,480))




while True:
    # Save the previous frame data
    previousGray = gray
     
    # Get the next frame
    ret , frame = cap.read()
    
    if ret:
        # Convert the frame to gray scale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray= cut_window(gray,[int(acumulator_x), int(acumulator_y)], frame)
        
        # Calculate the dense optical flow
        flow = cv2.calcOpticalFlowFarneback(previousGray, gray, None, 0.5, 3, 80, 3, 5, 1.2, 0)
        
        flow_height, flow_width, x =  flow.shape
        total_points = flow_height * flow_width
        
        x_counter = 0
        y_counter = 0
        
        for i in flow:
            for j in i:
                x_counter += j[0]
                y_counter += j[1]
            
        x_mean =  x_counter/total_points
        y_mean =  y_counter/total_points
        
        acumulator_x += x_mean
        acumulator_y +=  y_mean

        M = np.float32([[1,0,int(-acumulator_x)],[0,1,int(-acumulator_y)]])
        height,width,x = frame.shape
        
        estabilizado = cv.warpAffine(frame,M,(width,height))

        
        x_util = width - np.abs(acumulator_x)
        y_util =  height - np.abs(acumulator_y)
        
        proportion = width // height
        inverse_proportion =  height // width
        
        delta_y = 0
        delta_x = 0
        
        if proportion > x_util/y_util:
            delta_y =  -inverse_proportion*x_util + y_util
        else:
            delta_x = -proportion*y_util + x_util
            
        novo_x =  x_util - delta_x
        novo_y = y_util - delta_y
        print('n',novo_x)
        print('w',width)
        
        
        init_x_crop = None
        end_x_crop = None
        if(acumulator_x > 0 ):
            init_x_crop = 0
            end_x_crop = width - acumulator_x
        else:
            init_x_crop = np.abs(acumulator_x)
            end_x_crop = width
            
        init_y_crop = None
        end_y_crop = None
        if(acumulator_y > 0 ):
            init_y_crop = 0
            end_y_crop = height - acumulator_y
        else:
            init_y_crop = np.abs(acumulator_y)
            end_y_crop = height
        
        cropped = estabilizado[int(init_y_crop) : int(end_y_crop), int(init_x_crop) : int(end_x_crop)]

        
        final_img = cv.resize(cropped,(int(width), int(height)), novo_x/width, novo_y/height)


        output = final_img
        

        cv.imshow("Video", output)
        out.write(frame)
        out1.write(cropped)
        out2.write(estabilizado)
        out3.write(final_img)
        
        k = cv2.waitKey(30) & 0xff
        
        # Exit if the user press ESC
        if k == 27:
            break
    else:
        break

# When everything is done, release the capture and close all windows
cap.release()
out.release()
out1.release()
out2.release()
out3.release()
cv2.destroyAllWindows()


[0, 0]
[0, 0]
n 479.274096634
w 640
[0, 0]
n 478.633919878
w 640
[0, 1]
n 478.505827387
w 640
[0, -1]
n 478.555012066
w 640
[0, 1]
n 478.249462083
w 640
[0, -1]
n 479.143272457
w 640
[0, 0]
n 479.815082318
w 640
[0, 0]
n 478.456592136
w 640
[0, -1]
n 479.682478611
w 640
[0, 0]
n 477.095614446
w 640
[-1, -2]
n 475.515665269
w 640
[-2, -4]
n 477.148948806
w 640
[-7, -2]
n 475.170613143
w 640
[-12, -4]
n 477.744042981
w 640
[-21, -2]
n 476.662737673
w 640
[-19, -3]
n 478.281915178
w 640
[-27, -1]
n 476.439205898
w 640
[-29, -3]
n 478.485406073
w 640
[-35, -1]
n 476.628703123
w 640
[-37, -3]
n 476.608591911
w 640
[-45, -3]
n 477.60066636
w 640
[-44, -2]
n 477.173349342
w 640
[-53, -2]
n 476.963080117
w 640
[-49, -3]
n 479.434431347
w 640
[-56, 0]
n 476.658166002
w 640
[-55, -3]
n 479.563045927
w 640
[-63, 0]
n 478.070133324
w 640
[-61, -1]
n 479.15100742
w 640
[-70, 0]
n 478.775898205
w 640
[-67, 1]
n 477.907772351
w 640
[-73, 2]
n 477.851303049
w 640
[-68, 2]
n 477.363378944
w 640
[-70, 2